In [2]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d(1,6,5)
        self.conv2 = nn.Conv2d(6,16,5)
        self.fc1 = nn.Linear(16*5*5,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
        
    def forward(self,x):
        x=F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        x=F.max_pool2d(F.relu(self.conv2(x)),2)
        x=x.view(-1,self.num_flat_features(x))
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.fc3(x)
        return x
    
    def num_flat_features(self,x):
        size=x.size()[1:]
        num_features=1
        for s in size:
            num_features*=s
        return num_features

In [5]:
net=Net()

In [6]:
print(net)

Net (
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear (400 -> 120)
  (fc2): Linear (120 -> 84)
  (fc3): Linear (84 -> 10)
)


In [11]:
params=list(net.parameters())
print(len(params))

10


In [17]:
print(params[0].size())

torch.Size([6, 1, 5, 5])


In [18]:
input=Variable(torch.randn(1, 1, 32, 32))
out=net(input)
print(out)

Variable containing:
 0.1092 -0.0589 -0.0841  0.0763  0.0582  0.0385  0.0363  0.0026 -0.0791 -0.0531
[torch.FloatTensor of size 1x10]



In [19]:
net.zero_grad()
out.backward(torch.randn(1, 10))

In [20]:
######## Compute loss and backprop

In [23]:
output=net(input)
target=Variable(torch.arange(1,11)) #a dummy target variable
criterion = nn.MSELoss()

loss=criterion(output,target)
print(loss)

Variable containing:
 38.5846
[torch.FloatTensor of size 1]



In [25]:
loss

Variable containing:
 38.5846
[torch.FloatTensor of size 1]

In [26]:
print(loss.grad_fn)

In [27]:
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

In [28]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

In [29]:
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
Variable containing:
 0
 0
 0
 0
 0
 0
[torch.FloatTensor of size 6]

conv1.bias.grad after backward
Variable containing:
1.00000e-02 *
 -7.9508
 -5.6474
 -4.4492
  1.7173
 -2.1452
 -0.9283
[torch.FloatTensor of size 6]



In [30]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [34]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update

In [35]:
loss

Variable containing:
 37.4546
[torch.FloatTensor of size 1]